In [2]:
from transformers import AdamW,PreTrainedTokenizerFast
from torch.utils.data import Dataset, DataLoader
import torch
import json
import numpy as np
from tqdm import tqdm
import pandas as pd
import re

In [3]:
with open("./Similar_data.json", "r" , encoding="utf-8") as f:
    data = json.load(f)

print(len(data["data"]))

13279


In [4]:
print(data["data"][0]["para"][0]["label"][1]["sequence"])

담쟁이란 여름 한철 벽을 온통 둘러싸고 지붕과 연돌의 붉은 빛난 남기고 집 안을 통째로 초록의 세상으로 변해 줄 때가 아름다운 것이지, 잎을 다 떨어뜨리고 앙상하게 드러난 벽에 메마른 줄기를 깨알 같이 둘러칠 때쯤에는 벌써 다시 지릅떠볼 값조차 없는 것이다.


In [5]:
def multiple_replace(text, replacements):
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text

replacements = {"[CLS]": "", "[SEP]": ""}

context_label_pairs = []
count = 0
for item in data["data"]:
    for para in item["para"]:
        context = para["context"]
        for sequence in para["label"]:
            if str(type(sequence)) =="<class 'list'>":
                count +=1
                for sequence_list in sequence:
                    label = sequence_list["sequence"]
                    token_str = sequence_list["token_str"] 
                    label = label.replace("[MASK]",token_str)
                    label = multiple_replace(label,replacements)
                    context_label_pairs.append((context, label, token_str))
            else:
                label = sequence["sequence"]
                token_str = sequence["token_str"] 
                label =label.replace("[MASK]",token_str)
                label = multiple_replace(label,replacements)
                context_label_pairs.append((context, label, token_str))


print("count:",count)
for idx, (context,label, token_str) in enumerate(context_label_pairs[0:20],1):
    print(f"example{idx}:")
    print("context:", context)
    print("label", label)
    print("token_str:", token_str)

print(len(context_label_pairs))

count: 2849
example1:
context: 담쟁이란 여름 한철 벽을 온통 둘러싸고 지붕과 연돌의 붉은 빛난 남기고 집 안을 통째로 초록의 세상으로 변해 줄 때가 아름다운 것이지, 잎을 다 떨어뜨리고 앙상하게 드러난 벽에 메마른 줄기를 그물같이 둘러칠 때쯤에는 벌써 다시 지릅떠볼 값조차 없는 것이다.
label 담쟁이란 여름 한철 벽을 온통 둘러싸고 지붕과 연돌의 붉은 빛난 남기고 집 안을 통째로 초록의 세상으로 변해 줄 때가 아름다운 것이지, 잎을 다 떨어뜨리고 앙상하게 드러난 벽에 메마른 줄기를 산더미 같이 둘러칠 때쯤에는 벌써 다시 지릅떠볼 값조차 없는 것이다.
token_str: 산더미
example2:
context: 담쟁이란 여름 한철 벽을 온통 둘러싸고 지붕과 연돌의 붉은 빛난 남기고 집 안을 통째로 초록의 세상으로 변해 줄 때가 아름다운 것이지, 잎을 다 떨어뜨리고 앙상하게 드러난 벽에 메마른 줄기를 그물같이 둘러칠 때쯤에는 벌써 다시 지릅떠볼 값조차 없는 것이다.
label 담쟁이란 여름 한철 벽을 온통 둘러싸고 지붕과 연돌의 붉은 빛난 남기고 집 안을 통째로 초록의 세상으로 변해 줄 때가 아름다운 것이지, 잎을 다 떨어뜨리고 앙상하게 드러난 벽에 메마른 줄기를 깨알 같이 둘러칠 때쯤에는 벌써 다시 지릅떠볼 값조차 없는 것이다.
token_str: 깨알
example3:
context: 담쟁이란 여름 한철 벽을 온통 둘러싸고 지붕과 연돌의 붉은 빛난 남기고 집 안을 통째로 초록의 세상으로 변해 줄 때가 아름다운 것이지, 잎을 다 떨어뜨리고 앙상하게 드러난 벽에 메마른 줄기를 그물같이 둘러칠 때쯤에는 벌써 다시 지릅떠볼 값조차 없는 것이다.
label 담쟁이란 여름 한철 벽을 온통 둘러싸고 지붕과 연돌의 붉은 빛난 남기고 집 안을 통째로 초록의 세상으로 변해 줄 때가 아름다운 것이지, 잎을 다 떨어뜨리고 앙상하게 드러난 벽에 메마른 줄기를 다 같이 둘러칠 때쯤에는 벌써 다시 지릅떠볼 값조차 없는 것이다.
token_str: 다
exam

In [14]:
Modify_Similar_data = {"data":[]}
dict = {"data": [{"para": [{"label": [{}], "context": ""}]}]}

labels = {"label":[]}
previous_context = context_label_pairs[0][0]

for i,item in enumerate(context_label_pairs):
    para = {"para":[]}
    sequence = {}

    if previous_context != item[0] or i == len(context_label_pairs)-1:
        labels["context"] = previous_context
        previous_context = item[0]
        temp = labels
        para["para"].append(temp)
        Modify_Similar_data["data"].append(para)
        labels = {"label":[]}

    sequence["sequence"] = item[1]
    sequence["token_str"] = item[2]
    labels["label"].append(sequence)

  


In [16]:
len(Modify_Similar_data["data"])

11942

In [17]:
with open('./Similar_data_modify.json', 'w', encoding='utf-8') as file :
    json.dump(Modify_Similar_data, file, indent='\t', ensure_ascii=False)